In [ ]:
!pip install tensorflow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
import os

# Path to your zip file
zip_path = "/content/drive/MyDrive/plant_disease.zip"
extract_path = "/content/data"

shutil.unpack_archive(zip_path, extract_path)
print("Extraction done!")

# Base paths
base_dir = "/content/data/plant_disease_dataset"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")

# Remove old train/val folders if they exist
shutil.rmtree(train_dir, ignore_errors=True)
shutil.rmtree(val_dir, ignore_errors=True)

# Create fresh train/val folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


Extraction done!


In [ ]:
import random

source_dir = "/content/data/PlantVillage"  # Change if different
split_ratio = 0.8

for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        random.shuffle(images)

        split_index = int(len(images) * split_ratio)
        train_images = images[:split_index]
        val_images = images[split_index:]

        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
        for img in val_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

print("Images successfully copied to train and val folders!")


Images successfully copied to train and val folders!


In [ ]:
train_classes = set(os.listdir(train_dir))
val_classes = set(os.listdir(val_dir))

print("Classes in train but not in val:", train_classes - val_classes)
print("Classes in val but not in train:", val_classes - train_classes)

for cls in train_classes & val_classes:
    train_imgs = set(os.listdir(os.path.join(train_dir, cls)))
    val_imgs = set(os.listdir(os.path.join(val_dir, cls)))

    common_imgs = train_imgs & val_imgs
    if common_imgs:
        print(f"Some images are in both train and val for class '{cls}': {list(common_imgs)[:5]} ...")


Classes in train but not in val: set()
Classes in val but not in train: set()


In [ ]:
import os
import random
import shutil
from PIL import Image
from tqdm import tqdm

# Source dataset (your current dataset)
src_dir = '/content/data/plant_disease_dataset/train'
dst_dir = '/content/dataset_balanced/train'

# Desired number of images per class
TARGET_COUNT = 150  # You can pick 100 or 150

os.makedirs(dst_dir, exist_ok=True)

for class_name in os.listdir(src_dir):
    class_path = os.path.join(src_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    os.makedirs(os.path.join(dst_dir, class_name), exist_ok=True)

    if len(images) > TARGET_COUNT:
        # Undersample — randomly choose 150 images
        selected = random.sample(images, TARGET_COUNT)
    elif len(images) < TARGET_COUNT:
        # Oversample — duplicate until we reach 150
        selected = images.copy()
        while len(selected) < TARGET_COUNT:
            selected += random.sample(images, min(len(images), TARGET_COUNT - len(selected)))
    else:
        selected = images

    for img in tqdm(selected, desc=f"Copying {class_name}"):
        src_img = os.path.join(class_path, img)
        dst_img = os.path.join(dst_dir, class_name, img)
        if not os.path.exists(dst_img):
            shutil.copy(src_img, dst_img)

print("✅ Balanced dataset created at:", dst_dir)


Copying Tomato_Spider_mites_Two_spotted_spider_mite: 100%|██████████| 150/150 [00:00<00:00, 3647.41it/s]
Copying Tomato_Bacterial_spot: 100%|██████████| 150/150 [00:00<00:00, 6090.59it/s]

✅ Balanced dataset created at: /content/dataset_balanced/train


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dst_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 2221 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.
Number of classes: 15


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze first layers, unfreeze last 60 layers for better fine-tuning
for layer in base_model.layers[:-60]:
    layer.trainable = False
for layer in base_model.layers[-60:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)  # dropout to reduce overfitting
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2832 - loss: 2.3419
Epoch 1: val_accuracy improved from -inf to 0.40445, saving model to best_model.keras
70/70 ━━━━━━━━━━━━━━━━━━━━ 430s 6s/step - accuracy: 0.2856 - loss: 2.3345 - val_accuracy: 0.4045 - val_loss: 1.7882 - learning_rate: 1.0000e-04
Epoch 2/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7692 - loss: 0.7772
Epoch 2: val_accuracy did not improve from 0.40445
70/70 ━━━━━━━━━━━━━━━━━━━━ 424s 6s/step - accuracy: 0.7696 - loss: 0.7758 - val_accuracy: 0.3827 - val_loss: 1.9158 - learning_rate: 1.0000e-04
Epoch 3/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8594 - loss: 0.4528
Epoch 3: val_accuracy did not improve from 0.40445
70/70 ━━━━━━━━━━━━━━━━━━━━ 421s 6s/step - accuracy: 0.8596 - loss: 0.4523 - val_accuracy: 0.3667 - val_loss: 2.2549 - learning_rate: 1.0000e-04
Epoch 4/25
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9022 - loss: 0.3164
Epoch 4: val_accuracy improved from 0.40445 to 0

In [ ]:
model.save("/content/plant_disease_model1.keras")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy model
!cp /content/best_model.keras /content/drive/MyDrive/PlantDiseaseModel/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

path="/content/drive/MyDrive/PlantDiseaseModel/best_model.keras"
# -------------------------
# Load your previously trained model
# -------------------------
model = load_model(path)

In [ ]:
for layer in model.layers[:-60]:
    layer.trainable = False
for layer in model.layers[-60:]:
    layer.trainable = True

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # smaller LR to preserve learned weights
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [ ]:
checkpoint = ModelCheckpoint(
    'best_model_continued.keras',  # save to new file to preserve old model
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# -------------------------
# Continue training
# -------------------------
history = model.fit(
    train_generator,           # your training data generator
    validation_data=val_generator,  # your validation data generator
    epochs=15,                 # fewer epochs for fine-tuning
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 638ms/step - accuracy: 0.9246 - loss: 0.2513
Epoch 1: val_accuracy improved from -inf to 0.89115, saving model to best_model_continued.keras
70/70 ━━━━━━━━━━━━━━━━━━━━ 101s 972ms/step - accuracy: 0.9246 - loss: 0.2512 - val_accuracy: 0.8911 - val_loss: 0.3842 - learning_rate: 1.0000e-05
Epoch 2/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.9364 - loss: 0.1947
Epoch 2: val_accuracy improved from 0.89115 to 0.90252, saving model to best_model_continued.keras
70/70 ━━━━━━━━━━━━━━━━━━━━ 34s 480ms/step - accuracy: 0.9364 - loss: 0.1948 - val_accuracy: 0.9025 - val_loss: 0.3405 - learning_rate: 1.0000e-05
Epoch 3/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.9518 - loss: 0.1839
Epoch 3: val_accuracy improved from 0.90252 to 0.90687, saving model to best_model_continued.keras
70/70 ━━━━━━━━━━━━━━━━━━━━ 42s 489ms/step - accuracy: 0.9518 - loss: 0.1841 - val_accuracy: 0.9069 - val_loss: 0.3193 - learning_rate: 1.0000e-05
Epoch 4/1

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

path="/content/drive/MyDrive/best_model_continued.keras"
# -------------------------
# Load the previously trained model
# -------------------------
model = load_model(path)  # Replace with your saved model filename

# -------------------------
# Unfreeze last 60 layers for fine-tuning
# -------------------------
for layer in model.layers[:-60]:
    layer.trainable = False
for layer in model.layers[-60:]:
    layer.trainable = True

# -------------------------
# Recompile with smaller learning rate
# -------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # small LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -------------------------
# Callbacks (no continuous saving to speed up training)
# -------------------------
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True  # ensures the model ends up with best weights
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# -------------------------
# Continue training
# -------------------------
history = model.fit(
    train_generator,            # your training data
    validation_data=val_generator,  # your validation data
    epochs=15,                  # fewer epochs since model is already trained
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 97s 929ms/step - accuracy: 0.9522 - loss: 0.1639 - val_accuracy: 0.9475 - val_loss: 0.1639 - learning_rate: 1.0000e-05
Epoch 2/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 476ms/step - accuracy: 0.9457 - loss: 0.1452 - val_accuracy: 0.9465 - val_loss: 0.1679 - learning_rate: 1.0000e-05
Epoch 3/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 31s 450ms/step - accuracy: 0.9557 - loss: 0.1216 - val_accuracy: 0.9465 - val_loss: 0.1628 - learning_rate: 1.0000e-05
Epoch 4/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 468ms/step - accuracy: 0.9582 - loss: 0.1130 - val_accuracy: 0.9468 - val_loss: 0.1604 - learning_rate: 1.0000e-05
Epoch 5/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 32s 457ms/step - accuracy: 0.9511 - loss: 0.1334 - val_accuracy: 0.9458 - val_loss: 0.1602 - learning_rate: 1.0000e-05
Epoch 6/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 467ms/step - accuracy: 0.9609 - loss: 0.1215 - val_accuracy: 0.9470 - val_loss: 0.1571 - learning_rate: 1.0000e-05


In [ ]:
model.save("plant_disease_model_finetuned.keras")


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

path="/content/drive/MyDrive/plant_disease_model_finetuned.keras"
#  Load previously trained/best model
model = load_model(path)

#  Unfreeze more layers to allow deeper fine-tuning
for layer in model.layers:
    layer.trainable = True  # Unfreeze all layers (you can choose partial if needed)

#  Recompile after changing trainability
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for safety
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=7,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)
checkpoint = ModelCheckpoint(
    'plant_disease_model_finetuned.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Continue training
history2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,  # increase if you want deeper training
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

#  Save final model
model.save("plant_disease_model_finetuned.keras")
print("Model saved successfully as plant_disease_model_finetuned.keras")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 588ms/step - accuracy: 0.9077 - loss: 0.3059
Epoch 1: val_accuracy improved from -inf to 0.94678, saving model to plant_disease_model_finetuned.keras
128/128 ━━━━━━━━━━━━━━━━━━━━ 140s 705ms/step - accuracy: 0.9077 - loss: 0.3057 - val_accuracy: 0.9468 - val_loss: 0.1701 - learning_rate: 1.0000e-05
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.9303 - loss: 0.2231
Epoch 2: val_accuracy did not improve from 0.94678
128/128 ━━━━━━━━━━━━━━━━━━━━ 56s 439ms/step - accuracy: 0.9302 - loss: 0.2231 - val_accuracy: 0.9352 - val_loss: 0.1948 - learning_rate: 1.0000e-05
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.9359 - loss: 0.1867
Epoch 3: val_accuracy did not improve from 0.94678
128/128 ━━━━━━━━━━━━━━━━━━━━ 56s 440ms/step - accuracy: 0.9359 - loss: 0.1868 - val_accuracy: 0.9323 - val_loss: 0.2222 - learning_rate: 1.0000e-05
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step - accuracy: 0.9432 - loss: 0.1

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

path="/content/drive/MyDrive/plant_disease_model_finetuned.keras"
# Load previously trained/best model
model = load_model(path)

# Unfreeze more layers to allow deeper fine-tuning
for layer in model.layers:
    layer.trainable = True  # Unfreeze all layers (you can choose partial if needed)

# Recompile after changing trainability
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for safety
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=7,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)
checkpoint = ModelCheckpoint(
    'plant_disease_model_finetuned.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Continue training
history2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,  # increase if you want deeper training
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

#  Save final model
model.save("plant_disease_model_finetuned.keras")
print("Model saved successfully as plant_disease_model_finetuned.keras")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9108 - loss: 0.3025
Epoch 1: val_accuracy improved from -inf to 0.94388, saving model to plant_disease_model_finetuned.keras
70/70 ━━━━━━━━━━━━━━━━━━━━ 683s 9s/step - accuracy: 0.9110 - loss: 0.3017 - val_accuracy: 0.9439 - val_loss: 0.1837 - learning_rate: 1.0000e-05
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9360 - loss: 0.2023
Epoch 2: val_accuracy did not improve from 0.94388
70/70 ━━━━━━━━━━━━━━━━━━━━ 644s 9s/step - accuracy: 0.9360 - loss: 0.2024 - val_accuracy: 0.9342 - val_loss: 0.2066 - learning_rate: 1.0000e-05
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9285 - loss: 0.2126
Epoch 3: val_accuracy did not improve from 0.94388
70/70 ━━━━━━━━━━━━━━━━━━━━ 644s 9s/step - accuracy: 0.9286 - loss: 0.2126 - val_accuracy: 0.9313 - val_loss: 0.2229 - learning_rate: 1.0000e-05
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9487 - loss: 0.1603
Epoch 4: ReduceLROnPlateau re